In [1]:
!pip install tensorflow
!pip install librosa
!pip install scipy

You should consider upgrading via the 'c:\users\qkrwo\anaconda3\envs\nlp_task\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\qkrwo\anaconda3\envs\nlp_task\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\qkrwo\anaconda3\envs\nlp_task\python.exe -m pip install --upgrade pip' command.


# 1. Data Processing
우리는 데이터를 처리할 것이다.<br>
tfrecord 데이터의 label - s1, s2를 무지성으로 가져와 훈련시켜야 된다.<br>
label안에는 두 개의 데이터가 들어 있다. 바로 s1과 s2이다.<br>
따라서 batch * 2의 datasize가 될 것이며 우리는 그 해결법을 찾을 것이다.<br>
우선 causal하게 데이터를 생성하는 것이 필요하다.<br>
또한 sequence length에 padding을 두어서 길이를 조절할 필요성도 있다.

In [1]:
import tensorflow as tf
import string

import multiprocessing
import tensorflow as tf
import numpy as np
import librosa
from scipy import signal
import argparse
import os, sys
from numpy.fft import rfft, irfft
from scipy.io.wavfile import write as wav_write

def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
from functools import partial
import logging

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 25
INPUT_SIZE = 129
OUTPUT_SIZE = 129

In [4]:
startMask = tf.fill([1,129],-1)
endMask = tf.fill([1,129],-2)
print(startMask)
print(endMask)

tf.Tensor(
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1]], shape=(1, 129), dtype=int32)
tf.Tensor(
[[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
  -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
  -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
  -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
  -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
  -2 -2 -2 -2 -2 -2 -2 -2 -2]], shape=(1, 129), dtype=int32)


In [5]:
# Load the data

tr_path = './mycode/tfrecords/tr_tfrecord/*.tfrecords'
val_path = './mycode/tfrecords/cv_tfrecord/*.tfrecords'
tt_path = './mycode/tfrecords/tt_tfrecord/*.tfrecords'

FILENAMES_TRAINING = tf.io.gfile.glob(tr_path)
FILENAMES_VALIDATION = tf.io.gfile.glob(val_path)
FILENAMES_TEST = tf.io.gfile.glob(tt_path)

In [6]:

print("Train TFRecord Files:", len(FILENAMES_TRAINING))
print("Validation TFRecord Files:", len(FILENAMES_VALIDATION))
print("Test TFRecord Files:", len(FILENAMES_TEST))
FILENAMES_TRAINING

Train TFRecord Files: 4
Validation TFRecord Files: 4
Test TFRecord Files: 4


['.\\mycode\\tfrecords\\tr_tfrecord\\447o0302_0.62948_441c0212_-0.62948.tfrecords',
 '.\\mycode\\tfrecords\\tr_tfrecord\\447o0302_1.3388_22ho010i_-1.3388.tfrecords',
 '.\\mycode\\tfrecords\\tr_tfrecord\\447o0302_2.1067_422o030k_-2.1067.tfrecords',
 '.\\mycode\\tfrecords\\tr_tfrecord\\447o0303_0.14144_441c0212_-0.14144.tfrecords']

In [7]:
def data_preprocessing(data, check, input_size=129*2, output_size=129*2):
    if check == 'inputs':
        inputs = tf.slice(data, [0, 0], [-1, input_size//2])
        angle = tf.slice(data, [0, input_size//2], [-1, -1])
        
        return inputs, angle
    
    elif check == 'labels':
        label1 = tf.slice(data, [0, 0], [-1, output_size//2])
        label2 = tf.slice(data, [0, output_size//2], [-1, -1])
        
        return label1, label2

In [8]:
def data_preprocessing_causal(data, check, input_size=129*2, output_size=129*2):
    if check == 'inputs':
        label1 = tf.slice(data, [0, 0], [-1, input_size//2])
        label2 = tf.slice(data, [0, input_size//2], [-1, -1])
        
        return inputs, angle
    
    elif check == 'labels':
        label1 = tf.slice(data, [0, 0], [-1, output_size//2])
        label2 = tf.slice(data, [0, output_size//2], [-1, -1])
        
        return label1, label2

In [9]:
def read_tfrecord(example, input_size=129*2, output_size=129*2):
    tfrecord_format = (
        {
            'inputs': tf.io.FixedLenSequenceFeature(shape=[input_size], dtype=tf.float32, allow_missing=True),
            'labels': tf.io.FixedLenSequenceFeature(shape=[output_size], dtype=tf.float32, allow_missing=True),
            'genders': tf.io.FixedLenSequenceFeature(shape=[2], dtype=tf.float32, allow_missing=True)
        }
    )
    _, example = tf.io.parse_single_sequence_example(example, sequence_features=tfrecord_format)
    inputs, angle = data_preprocessing(example["inputs"], 'inputs', input_size)
#     label1, label2 = data_preprocessing(example["labels"], 'labels', input_size)
    
    return inputs, example['labels']

In [10]:
def read_tfrecord_test(example, input_size=129*2, output_size=129*2):
    tfrecord_format = (
        {
            'inputs': tf.io.FixedLenSequenceFeature(shape=[input_size], dtype=tf.float32, allow_missing=True),
            'labels': tf.io.FixedLenSequenceFeature(shape=[output_size], dtype=tf.float32, allow_missing=True),
            'genders': tf.io.FixedLenSequenceFeature(shape=[2], dtype=tf.float32, allow_missing=True)
        }
    )
    _, example = tf.io.parse_single_sequence_example(example, sequence_features=tfrecord_format)
    inputs, angle = data_preprocessing(example["inputs"], 'inputs', input_size)
    
    return inputs, angle, example['labels']

In [11]:
def load_dataset(filenames, input_size=129*2, output_size=129*2, check='train'):
    ignore_order = tf.data.Options()
    
    if check == 'train':
        ignore_order.experimental_deterministic = False  # disable order, increase speed
    else:
        ignore_order.experimental_deterministic = True
    
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    
    if check == 'train':
        dataset = dataset.map(
            partial(read_tfrecord, input_size=input_size, output_size=output_size), num_parallel_calls=AUTOTUNE
        )
    else:
        dataset = dataset.map(
            partial(read_tfrecord_test, input_size=input_size, output_size=output_size), num_parallel_calls=AUTOTUNE
        )
    
    return dataset

In [12]:
def get_dataset(filenames, input_size=129*2, output_size=129*2):
    dataset = load_dataset(filenames, input_size, output_size)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=(None))
    
    return dataset

In [13]:
def get_dataset_for_test(filenames, input_size=129*2, output_size=129*2):
    dataset = load_dataset(filenames, input_size, output_size, check='test')
    dataset = dataset.repeat(1)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=(None))
    
    return dataset

In [14]:
train_dataset = get_dataset(FILENAMES_TRAINING, INPUT_SIZE*2, OUTPUT_SIZE*2)
valid_dataset = get_dataset(FILENAMES_VALIDATION, INPUT_SIZE*2, OUTPUT_SIZE*2)

test_dataset = get_dataset_for_test(FILENAMES_TEST, INPUT_SIZE*2, OUTPUT_SIZE*2)

In [15]:
for inp, lbl in train_dataset:
    print(inp.numpy().shape)
    print(lbl.numpy().shape)

(4, 583, 129)
(4, 583, 258)


# 2. Building model
- 기존꺼 썼으니 이제는 T5의 시대이다 시발

In [16]:
import threading, sys
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Reshape, LSTM, Concatenate, Multiply, Bidirectional, Dropout
from tensorflow.keras import optimizers
from tensorflow import keras

import scipy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [17]:
BATCH_SIZE = 25
INPUT_SIZE = 129
OUTPUT_SIZE = 129

mkdir_p('./CKPT/') # model check point 폴더 만드는 코드
filepath = "./CKPT/CKP_ep_{epoch:d}__loss_{val_loss:.5f}_.h5"

In [18]:
from tensorflow.keras.layers import LayerNormalization, ReLU, Embedding
from collections import namedtuple

Config = namedtuple('Config',
  field_names="d_ff,     d_kv,     d_model,              dropout, feed_forward_activation, num_layers," 
              "layer_norm_epsilon, model_type, num_heads, positional_embedding, n_epochs, vocab_size,"
                "model_path, wav_type, size_type, train_type, loss_type, learning_rate_type")
args = Config( 2048      , 64      , 512              , 0.1 , "gated_gelu", 4       , 
               1e-06    , "t5"             , 8 , "absolute" , 4     , 129   ,
             "CKPT", "wav8k", "min", "train-360", "mse", "inverse_root")

T5Config {
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layer": 2,
  "num_layers": 6,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.7.0.dev0",
  "use_cache": true,
  "vocab_size": 32128
}

Config = namedtuple('Config',
  field_names="embed_dim, hidden_dim, num_max_positions, num_embeddings      , num_heads, num_layers," 
              "dropout, initializer_range, batch_size, lr, max_norm, n_epochs, n_warmup,"
              "mlm, gradient_accumulation_steps, device, log_dir, dataset_cache")
args = Config( 128      , 1024      , 256              , 129, 8       , 16        ,
               0.1    , 0.02             , 16        , 2.5e-4, 1.0 , 10     , 1000    ,
               False, 4, "cuda" if torch.cuda.is_available() else "cpu", "./"   , "./dataset_cache.bin")

In [19]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [20]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [21]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq,0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :] # (batch_size, 1, 1, seq_len, emb)

In [22]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [23]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

# our model

def T5(input_size, output_size, batch, config):
    inputs = Input(shape=(None, input_size))
    
    # Embedding but STTF uses dense layer for embedding
    x = Dense(config.d_model, activation = 'tanh')(inputs) # input
    
    
    # ------------------- Encoder ----------------
    y = MultiHeadAttention(config.d_model, config.num_heads)(x, k=x, q=x, mask=None)
    # Self Attention + relative attention bias
    
    # skip connection
    layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    # layer normalization
    # dropout
    
    y = Dense_GatedGelu_Dense(config.d_model, config.d_ff, config.dropout)(y)
    # Dense
    # RELU # , config.feed_forward_activation
    # dropout
    # Dense
    
    # * layer_num Encoder
    
    # -------------------- Decoder ---------------
    # Self Attention + relative attention bias
    # skip connection
    # layer normalization
    # dropout
    
    # Layer Cross Attention
    # skip connection
    # layer normalization
    # dropout
    
    # Dense
    # RELU
    # deopout
    # Dense
    
    # * layer_num Decoder
    
    
    outputs = Bidirectional(LSTM(496, activation = 'tanh', return_sequences=True),
                           input_shape=(None, 496,))(outputs)
    drop = Dropout(rate=0.8)(outputs)
    outputs = Bidirectional(LSTM(496, activation = 'tanh', return_sequences=True))(drop)
    drop = Dropout(rate=0.8)(outputs)
    outputs = Bidirectional(LSTM(496, activation = 'tanh', return_sequences=True))(drop)
    drop = Dropout(rate=0.8)(outputs)
    
    pred1 = Dense(output_size, activation = 'relu')(drop)
    pred2 = Dense(output_size, activation = 'relu')(drop)
    
    cleaned1 = Multiply()([pred1, inputs])
    cleaned2 = Multiply()([pred2, inputs])
    
    model = Concatenate()([cleaned1, cleaned2])
    
    model = keras.Model(inputs, model)
    
    model.summary()
    adam = tf.optimizers.Adam(learning_rate=0.001)
    #model.compile(loss=pit_with_outputsize(output_size), optimizer=adam)
    model.compile(loss=keras.losses.mean_squared_error, optimizer=adam)

    return model

In [24]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads # 512/8 = 64

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [25]:
def dense_relu_dense(d_model, d_ff, dropout = 0.1): 
    return tf.keras.Sequential([
        tf.keras.layers.Dense(d_ff, activation='relu', use_bias = False),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dropout(dropout), # (batch_size, seq_len, d_model)
        tf.keras.layers.Dense(d_model, use_bias = False)  # (batch_size, seq_len, d_model)
    ])

In [26]:
class Dense_GatedGelu_Dense(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff, dropout = 0.1):
        super(Dense_GatedGelu_Dense, self).__init__()
        self.dense_gelu = tf.keras.layers.Dense(d_ff, activation='relu', use_bias = False)  # (batch_size, seq_len, dff)
        self.linear = tf.keras.layers.Dense(d_ff, use_bias = False)
        self.add_gates = tf.keras.layers.Add()
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.fin_dense = tf.keras.layers.Dense(d_model, use_bias = False)
        
    def call(self, hidden_states):
        hidden_gelu= self.dense_gelu(hidden_states)
        hidden_linear = self.linear(hidden_states)
        hidden_states = self.add_gates([hidden_gelu, hidden_linear])
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.fin_dense(hidden_states)
        
        return hidden_states


In [27]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = Dense_GatedGelu_Dense(d_model, d_ff, dropout)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, x, training, mask):

        normed_x = self.layernorm1(x)
        attn_output, _ = self.mha(normed_x, normed_x, normed_x, mask)  # (batch_size, input_seq_len, d_model)
        out1 = x + self.dropout1(attn_output, training=training)  # (batch_size, input_seq_len, d_model)

        normed_out1 = self.layernorm2(out1)
        ffn_output = self.ffn(normed_out1)  # (batch_size, input_seq_len, d_model)
        out2 = out1 + self.dropout2(ffn_output, training=training)  # (batch_size, input_seq_len, d_model)
        
        return out2

In [28]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = Dense_GatedGelu_Dense(d_model, d_ff, dropout)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)
        self.dropout3 = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        normed_x = self.layernorm1(x)
        print('normed_x', normed_x.shape)
        attn1, attn_weights_block1 = self.mha1(normed_x, normed_x, normed_x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        print('attn1', attn1.shape)
        out1 = x + self.dropout1(attn1, training=training)

        normed_out1 = self.layernorm2(out1)
        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, normed_out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        print('attn2', attn2.shape)
        attn2 = self.dropout2(attn2, training=training)
        out2 = attn2 + out1  # (batch_size, target_seq_len, d_model)

        normed_out2 = self.layernorm3(out2)
        ffn_output = self.ffn(normed_out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        print('ffn_output', ffn_output.shape)
        out3 = ffn_output + out2  # (batch_size, target_seq_len, d_model)
        print('out3', out3.shape)

        return out3, attn_weights_block1, attn_weights_block2

In [29]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, d_ff, input_size,
               maximum_position_encoding, dropout=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.embedding = tf.keras.layers.Dense(self.d_model, activation = 'tanh') # For STFT Input 
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, d_ff, dropout)
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, training, mask):
        print('x : ',x)

        seq_len = tf.shape(x)[1]
        print('seq_len : ',seq_len)

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        print('embedded : ',x.shape)
        
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [30]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, d_ff, target_size, maximum_position_encoding, dropout=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        #self.embedding = tf.keras.layers.Embedding(target_size, d_model)
        self.embedding = tf.keras.layers.Dense(self.d_model, activation = 'tanh') # For STFT Input 
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_output, training,look_ahead_mask, padding_mask):

        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                                 look_ahead_mask, padding_mask)

            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

In [55]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, d_ff, input_size,
               target_size, pe_input, pe_target, dropout=0.1):
        super(Transformer, self).__init__()

        self.tokenizer = Encoder(num_layers, d_model, num_heads, d_ff,
                                 input_size, pe_input, dropout)

        self.decoder = Decoder(num_layers, d_model, num_heads, d_ff,
                               target_size, pe_target, dropout)

        self.final_layer = tf.keras.layers.Dense(target_size)
        
        #self.dropout = Dropout(dropout)
        
        #self.maskLayer1 = Dense(input_size, activation = 'relu')
        #self.maskLayer2 = Dense(input_size, activation = 'relu')
        
        self.mtp = Multiply()
        #self.mtp2 = Multiply()
        
        self.concat = Concatenate()

    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        # inp (4, 583, 129) -> *2 = (3, 583, 256)
        # tar (4, 584, 129)
        print('input : ',inp.shape)
        enc_output = self.tokenizer(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
        print('enc_output',enc_output.shape)

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        print('target : ',tar.shape)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)
        print('dec_output',enc_output.shape)
        
        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
        #inputs = tf.concat([inp,inp],2)
        #print('mask_output',final_output.shape)
        
        #final_output = self.mtp([final_output[:,:-1,:], inputs])
        print('final_output',final_output.shape)
        # added Layers for Speech Separation
        
        inputs = self.concat([inp[:,:tar.shape[1],:], inp[:,:tar.shape[1],:]])
        
        final_output = self.mtp([final_output, inputs])
        """
        final_output = self.dropout(final_output, training = training)
        pred1 = self.maskLayer1(final_output)
        pred2 = self.maskLayer2(final_output)
        
        cleaned1 = Multiply()([pred1, inp])
        cleaned2 = Multiply()([pred2, inp])
        
        final_output = Concatenate()([cleaned1, cleaned2])
        """
        return final_output, attention_weights

In [56]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [57]:
# our model

def T5(input_size, output_size, batch, config):
    inputs = Input(shape=(None, input_size))
    targets = Input(shape=(None, input_size*2))
    
    # Embedding but STTF uses dense layer for embedding
    #x = Dense(config.d_model, activation = 'tanh')(inputs) # input
    
    #y = EncoderLayer(config.d_model, config.num_heads, config.d_ff, config.dropout)(x,training=False, mask=None) # d_model, num_heads, d_ff, dropout=0.1
    # ------------------- Encoder ----------------
    #y, attention_weights = MultiHeadAttention(config.d_model, config.num_heads)(x, k=x, q=x, mask=None)
    # Self Attention + relative attention bias
    
    # skip connection
    # layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    # layer normalization
    # dropout
    
    #y = Dense_GatedGelu_Dense(config.d_model, config.d_ff, config.dropout)(y)
    # Dense
    # RELU # , config.feed_forward_activation
    # dropout
    # Dense
    
    # * layer_num Encoder
    sample_transformer = Transformer(num_layers=config.num_layers, d_model=config.d_model, num_heads=config.num_heads, d_ff=config.d_ff,
        input_size=input_size, target_size=258,
        pe_input=10000, pe_target=6000, dropout = config.dropout)
    outputs, attention_weights = sample_transformer(inputs, targets, training=False,
                               enc_padding_mask=None,
                               look_ahead_mask=None,
                               dec_padding_mask=None) # (batch_size, tar_seq_len, target_vocab_size)
    
    
    # for Fine-Tuning
    """
    pred1 = Dense(output_size, activation = 'relu')(outputs)
    pred2 = Dense(output_size, activation = 'relu')(outputs)
    
    cleaned1 = Multiply()([pred1, inputs])
    cleaned2 = Multiply()([pred2, inputs])
    
    model = Concatenate()([cleaned1, cleaned2])
    """
    # -------------------- Decoder ---------------
    # Self Attention + relative attention bias
    # skip connection
    # layer normalization
    # dropout
    
    # Layer Cross Attention
    # skip connection
    # layer normalization
    # dropout
    
    # Dense
    # RELU
    # deopout
    # Dense
    
    # * layer_num Decoder
    model = keras.Model([inputs, targets], outputs)
    
    model.summary()
    learning_rate = CustomSchedule(config.d_model, warmup_steps=4000)
    adam = tf.optimizers.Adam(learning_rate=learning_rate)
    #model.compile(loss=pit_with_outputsize(output_size), optimizer=adam)
    model.compile(loss=keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM), optimizer=adam)

    return model

In [58]:
model = T5(INPUT_SIZE, OUTPUT_SIZE, BATCH_SIZE, args)

input :  (None, None, 129)
x :  Tensor("Placeholder:0", shape=(None, None, 129), dtype=float32)
seq_len :  Tensor("transformer_1/encoder_1/strided_slice:0", shape=(), dtype=int32)
embedded :  (None, None, 512)
enc_output (None, None, 512)
target :  (None, None, 258)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
dec_output (None, None, 512)
final_output (None, None, 258)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape       

In [59]:
loss_object = tf.keras.losses.MeanSquaredError(
    reduction=tf.keras.losses.Reduction.AUTO, name='mean_squared_error'
)
#tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    #mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    #mask = tf.cast(mask, dtype=loss_.dtype)
    #loss_ *= mask

    return tf.reduce_sum(loss_)#/tf.reduce_sum(mask)


def accuracy_function(real, pred):
    accuracies = tf.equal(real, tf.argmax(pred, axis=2))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [60]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [61]:
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask

# 3. Training ^^
무지성으로 훈련시키자. future work로 좀 더 해결해야할 사항이 남아있음.

In [62]:
checkpoint_path = "./checkpoints/train"

learning_rate = CustomSchedule(args.d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
transformer = Transformer(num_layers=args.num_layers, d_model=args.d_model, num_heads=args.num_heads, d_ff=args.d_ff,
        input_size=129, target_size=258,
        pe_input=10000, pe_target=6000, dropout = args.dropout)

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!!')

In [63]:
startMask = tf.cast(startMask,dtype=tf.float32)
endMask = tf.cast(endMask,dtype=tf.float32)

In [64]:
train_step_signature = [
    tf.TensorSpec(shape=(None, None, 129), dtype=tf.float32),
    tf.TensorSpec(shape=(None, None, 258), dtype=tf.float32),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar): # (batch, seq_len, values)
    """print('inp',inp.shape) 
    startMask = tf.cast(tf.fill([1,258],-1),dtype=tf.float32)
    endMask = tf.cast(tf.fill([1,258],-2),dtype=tf.float32)
    tar_inp = tf.concat([startMask, tar],0)
    tar_real = tf.concat([tar, endMask],0)
    print('tar_inp',tar_inp.shape)
    print('tar_real',tar_real[0])"""
    tar_inp = tar[:, :-1, :]
    tar_real = tar[:, 1:, :]
    print('tar_inp',tar_inp.shape)

    #enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
    enc_padding_mask, combined_mask, dec_padding_mask = None, None, None
    combined_mask = create_look_ahead_mask(tf.shape(tar_inp)[1])
    combined_mask = combined_mask[tf.newaxis, tf.newaxis, :, :]
    combined_mask = tf.repeat(combined_mask, tf.shape(tar_inp)[0], 0)
    
    print(enc_padding_mask, combined_mask, dec_padding_mask)
    with tf.GradientTape() as tape:
        predictions, _ = transformer(inp, tar_inp,
                                     True,
                                     enc_padding_mask,
                                     combined_mask,
                                     dec_padding_mask)
        
        loss = loss_function(tar_real, predictions)
        
    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    print('end_first')
    #train_accuracy(accuracy_function(tar_real, predictions))

In [65]:
import time

for epoch in range(10):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    for (batch, (inp, tar)) in enumerate(train_dataset):
        startMask = tf.cast(tf.fill([tar[:,:].shape[0], 1,258],-1),dtype=tf.float32)
        #endMask = tf.cast(tf.fill([tar[:,:].shape[0], 1,258],-2),dtype=tf.float32)
        tar = tf.concat([startMask, tar],1)
        #tar = tf.concat([tar,endMask],1)
        
        train_step(inp, tar)

        if batch % 50 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

    print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f}')

    print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

tar_inp (None, None, 258)
None Tensor("Repeat/Reshape_1:0", shape=(None, 1, None, None), dtype=float32) None
input :  (None, None, 129)
x :  Tensor("inp:0", shape=(None, None, 129), dtype=float32)
seq_len :  Tensor("transformer_2/encoder_2/strided_slice:0", shape=(), dtype=int32)
embedded :  (None, None, 512)
enc_output (None, None, 512)
target :  (None, None, 258)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
dec_output (None, None, 512)
final_output (None, None, 258)
end_first
tar_inp (None, None, 258)
None Tensor("Repeat/Reshape_

In [73]:
def evaluate(inp, max_length=1800):
    # inp sentence is portuguese, hence adding the start and end token
    encoder_input = inp
    max_length = inp.shape.as_list()[1]
    batch_size = inp.shape.as_list()[0]
    
    # as the target is english, the first word to the transformer should be the
    # english start token.
    startMask = tf.cast(tf.fill([1,258],-1),dtype=tf.float32)
    output = tf.expand_dims(startMask, 0)
    output = tf.repeat(output, batch_size, 0)
    zero_clipping = tf.constant([0.])
    print("output:",output)

    for i in range(max_length):
        enc_padding_mask, combined_mask, dec_padding_mask = None, None, None
        combined_mask = create_look_ahead_mask(tf.shape(output)[1])
        combined_mask = combined_mask[tf.newaxis, tf.newaxis, :, :]
        combined_mask = tf.repeat(combined_mask, tf.shape(output)[0], 0)

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(encoder_input,
                                                     output,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)

        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        predictions = tf.math.maximum(predictions, zero_clipping)
        predicted_id = predictions

        # concatentate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=1)

    return output, attention_weights

In [ ]:
sentence = "este é um problema que temos que resolver."
ground_truth = "this is a problem we have to solve ."

translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

In [ ]:
for (batch, (inp, tar)) in enumerate(train_dataset):
    startMask = tf.cast(tf.fill([tar[:,:].shape[0], 1,258],-1),dtype=tf.float32)
    endMask = tf.cast(tf.fill([tar[:,:].shape[0], 1,258],-2),dtype=tf.float32)
    print(tf.concat([tar,endMask],1)[0,-1])
    print( tf.concat([inp,inp],2).shape)
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

In [567]:
for (batch, (inp, tar)) in enumerate(train_dataset):
    print(inp.numpy().shape)
    print(tar.numpy().shape)
    print(inp[1].numpy().shape)
    print(inp[1].numpy().shape)    
    print(inp[2].numpy().shape)    
    print(inp[3].numpy().shape)    
    print(tar[0].numpy().shape)
    print(tar[1].numpy().shape)    
    print(tar[2].numpy().shape)    
    print(tar[3].numpy().shape)
    startMask = tf.cast(tf.fill([tar[:,:].shape[0], 1,258],-1),dtype=tf.float32)
    endMask = tf.cast(tf.fill([tar[:,:].shape[0], 1,258],-2),dtype=tf.float32)
    tar = tf.concat([startMask, tar],1)
    tar = tf.concat([tar,endMask],1)
    tar_inp = tar[:, :-1, :]
    tar_real = tar[:, 1:, :]
    print(tar_inp.numpy().shape)
    print(tar_real.numpy().shape)
    tests = create_look_ahead_mask(tf.shape(tar)[1])
    tests = tests[tf.newaxis, tf.newaxis, :, :]
    tests = tf.repeat(tests, 4, 0)
    print(tests)

(4, 583, 129)
(4, 583, 258)
(583, 129)
(583, 129)
(583, 129)
(583, 129)
(583, 258)
(583, 258)
(583, 258)
(583, 258)
(4, 584, 258)
(4, 584, 258)
tf.Tensor(
[[[[0. 1. 1. ... 1. 1. 1.]
   [0. 0. 1. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 1.]
   ...
   [0. 0. 0. ... 0. 1. 1.]
   [0. 0. 0. ... 0. 0. 1.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 1. 1. ... 1. 1. 1.]
   [0. 0. 1. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 1.]
   ...
   [0. 0. 0. ... 0. 1. 1.]
   [0. 0. 0. ... 0. 0. 1.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 1. 1. ... 1. 1. 1.]
   [0. 0. 1. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 1.]
   ...
   [0. 0. 0. ... 0. 1. 1.]
   [0. 0. 0. ... 0. 0. 1.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 1. 1. ... 1. 1. 1.]
   [0. 0. 1. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 1.]
   ...
   [0. 0. 0. ... 0. 1. 1.]
   [0. 0. 0. ... 0. 0. 1.]
   [0. 0. 0. ... 0. 0. 0.]]]], shape=(4, 1, 585, 585), dtype=float32)


In [630]:
# validation loss에 대해서 좋은 것만 저장됨
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'
)

# early stop 하는 부분인데, validation loss에 대해서 제일 좋은 모델이 저장됨
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True
)

In [631]:
# Training part

epoch = args.n_epochs

strategy = tf.distribute.MirroredStrategy()
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    # 사용 안할 때는 load_model 주석 처리 하자
#     model = load_model('./CKPT/CKP_ep_29__loss_102.63367_.h5', custom_objects={'pit_loss': pit_with_outputsize(OUTPUT_SIZE)})
    
    model = T5(INPUT_SIZE, OUTPUT_SIZE, BATCH_SIZE, args)
    tf.executing_eagerly()

history = model.fit(
    train_dataset,
    epochs=epoch,
    validation_data=valid_dataset,
    shuffle=True,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
input :  (None, None, 129)
x :  Tensor("Placeholder:0", shape=(None, None, 129), dtype=float32)
seq_len :  Tensor("transformer_63/encoder_63/strided_slice:0", shape=(), dtype=int32)
embedded :  (None, None, 512)
enc_output (None, None, 512)
target :  (None, None, 129)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
normed_x (None, None, 512)
attn1 (None, None, 512)
attn2 (None, None, 512)
ffn_output (None, None, 512)
out3 (None, None, 512)
dec_output (None, None, 512)
mask_output (None, None, 258)
final_output (None, None, 258)
Model: "model_25"
____

ValueError: in user code:

    c:\users\qkrwo\anaconda3\envs\nlp_task\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    c:\users\qkrwo\anaconda3\envs\nlp_task\lib\site-packages\tensorflow\python\keras\engine\training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\qkrwo\anaconda3\envs\nlp_task\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\qkrwo\anaconda3\envs\nlp_task\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\qkrwo\anaconda3\envs\nlp_task\lib\site-packages\tensorflow\python\distribute\mirrored_strategy.py:679 _call_for_each_replica
        self._container_strategy(), fn, args, kwargs)
    c:\users\qkrw

In [42]:
sample_rate = 8000
window_size = 256
window_shift = 128

In [43]:
mkdir_p('./test_wav/') # Result wav 폴더 만드는 코드

In [44]:
def _biorthogonal_window_loopy(analysis_window, shift):
    """
    This version of the synthesis calculation is as close as possible to the
    Matlab impelementation in terms of variable names.
    The results are equal.
    The implementation follows equation A.92 in
    Krueger, A. Modellbasierte Merkmalsverbesserung zur robusten automatischen
    Spracherkennung in Gegenwart von Nachhall und Hintergrundstoerungen
    Paderborn, Universitaet Paderborn, Diss., 2011, 2011
    """
    fft_size = len(analysis_window)
    assert np.mod(fft_size, shift) == 0
    number_of_shifts = len(analysis_window) // shift

    sum_of_squares = np.zeros(shift)
    for synthesis_index in range(0, shift):
        for sample_index in range(0, number_of_shifts + 1):
            analysis_index = synthesis_index + sample_index * shift

            if analysis_index + 1 < fft_size:
                sum_of_squares[synthesis_index] \
                    += analysis_window[analysis_index] ** 2

    sum_of_squares = np.kron(np.ones(number_of_shifts), sum_of_squares)
    synthesis_window = analysis_window / sum_of_squares / fft_size
    return synthesis_window

In [45]:
def istft(stft_signal, size=1024, shift=256,
          window=signal.blackman, fading=True, window_length=None):
    """
    Calculated the inverse short time Fourier transform to exactly reconstruct
    the time signal.
    :param stft_signal: Single channel complex STFT signal
        with dimensions frames times size/2+1.
    :param size: Scalar FFT-size.
    :param shift: Scalar FFT-shift. Typically shift is a fraction of size.
    :param window: Window function handle.
    :param fading: Removes the additional padding, if done during STFT.
    :param window_length: Sometimes one desires to use a shorter window than
        the fft size. In that case, the window is padded with zeros.
        The default is to use the fft-size as a window size.
    :return: Single channel complex STFT signal
    :return: Single channel time signal.
    """
    assert stft_signal.shape[1] == size // 2 + 1

    if window_length is None:
        window = window(size)
    else:
        window = window(window_length)
        window = np.pad(window, (0, size - window_length), mode='constant')

    window = _biorthogonal_window_loopy(window, shift)

    # Why? Line created by Hai, Lukas does not know, why it exists.
    window *= size
    time_signal = scipy.zeros(stft_signal.shape[0] * shift + size - shift)

    for j, i in enumerate(range(0, len(time_signal) - size + shift, shift)):
        time_signal[i:i + size] += window * np.real(irfft(stft_signal[j]))

    # Compensate fade-in and fade-out
    if fading:
        time_signal = time_signal[size - shift:len(time_signal) - (size - shift)]

    return time_signal

In [46]:
def audiowrite(data, path, samplerate=16000, normalize=False, threaded=True):
    """ Write the audio data ``data`` to the wav file ``path``
    The file can be written in a threaded mode. In this case, the writing
    process will be started at a separate thread. Consequently, the file will
    not be written when this function exits.
    :param data: A numpy array with the audio data
    :param path: The wav file the data should be written to
    :param samplerate: Samplerate of the audio data
    :param normalize: Normalize the audio first so that the values are within
        the range of [INTMIN, INTMAX]. E.g. no clipping occurs
    :param threaded: If true, the write process will be started as a separate
        thread
    :return: The number of clipped samples
    """
    data = data.copy()
    int16_max = np.iinfo(np.int16).max
    int16_min = np.iinfo(np.int16).min

    if normalize:
        if not data.dtype.kind == 'f':
            data = data.astype(np.float)
        data /= np.max(np.abs(data))

    if data.dtype.kind == 'f':
        data *= int16_max

    sample_to_clip = np.sum(data > int16_max)
    if sample_to_clip > 0:
        print('Warning, clipping {} samples'.format(sample_to_clip))
    data = np.clip(data, int16_min, int16_max)
    data = data.astype(np.int16)

    if threaded:
        threading.Thread(target=wav_write, args=(path, samplerate, data)).start()
    else:
        wav_write(path, samplerate, data)

    return sample_to_clip

In [77]:

with tf.device('/cpu:0'):
    cnt = 0
    for batch in test_dataset:
        input_batch, angle_batch, label_batch = batch
        tf.executing_eagerly() # requires r1.7
        angle_numpy = tf.constant(angle_batch)
        angle_numpy = angle_numpy.numpy()
        
        result, _ = evaluate(input_batch, max_length=1800)
        result = result[:,1:,:]
        label1 = tf.slice(result, [0, 0, 0], [-1, -1, OUTPUT_SIZE])
        label2 = tf.slice(result, [0, 0, OUTPUT_SIZE], [-1, -1, -1])
        spec1 = label1 * np.exp(angle_numpy * 1j)
        spec2 = label2 * np.exp(angle_numpy * 1j)

        num = cnt * BATCH_SIZE
        for i in range(BATCH_SIZE):
            if i >= input_batch.shape[0]:
                sys.exit(0)
            
            wav_name1 = './test_wav/' + str(i + num) + 'th_1.wav'
            wav_name2 = './test_wav/' + str(i + num) + 'th_2.wav'
            wav1 = istft(spec1[i, 0:input_batch[i].shape[0], :], size=window_size, shift=window_shift)
            wav2 = istft(spec2[i, 0:input_batch[i].shape[0], :], size=window_size, shift=window_shift)
            audiowrite(wav1, wav_name1, sample_rate, True, True)
            audiowrite(wav2, wav_name2, sample_rate, True, True)

        if (cnt + 1) % 10 == 0:
            print((cnt + 1) * BATCH_SIZE)

        cnt += 1

output: tf.Tensor(
[[[-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]]

 [[-1. -1. -1. ... -1. -1. -1.]]], shape=(4, 1, 258), dtype=float32)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 ... 1.2616850e-02
   1.1365835e-02 1.0914088e-02]
  [1.7649380e-03 1.7621844e-03 1.7541876e-03 ... 2.5867332e-05
   1.9198062e-05 1.6374195e-05]]

 [[1.3797548e-02 1.1909432e-02 7.6560592e-03 ... 1.1418096e-03
   1.0196227e-03 1.2328980e-03]
  [3.1644858e-02 1.9962942e-02 7.2805141e-03 ... 2.2434427e-03
   1.4804254e-03 7.6991477e-04]
  [

enc_output (4, 583, 512)
target :  (4, 3, 258)
normed_x (4, 3, 512)
attn1 (4, 3, 512)
attn2 (4, 3, 512)
ffn_output (4, 3, 512)
out3 (4, 3, 512)
normed_x (4, 3, 512)
attn1 (4, 3, 512)
attn2 (4, 3, 512)
ffn_output (4, 3, 512)
out3 (4, 3, 512)
normed_x (4, 3, 512)
attn1 (4, 3, 512)
attn2 (4, 3, 512)
ffn_output (4, 3, 512)
out3 (4, 3, 512)
normed_x (4, 3, 512)
attn1 (4, 3, 512)
attn2 (4, 3, 512)
ffn_output (4, 3, 512)
out3 (4, 3, 512)
dec_output (4, 583, 512)
final_output (4, 3, 258)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 ... 1.2616850e-02
   1.1365835e-02 1.0914088e

enc_output (4, 583, 512)
target :  (4, 6, 258)
normed_x (4, 6, 512)
attn1 (4, 6, 512)
attn2 (4, 6, 512)
ffn_output (4, 6, 512)
out3 (4, 6, 512)
normed_x (4, 6, 512)
attn1 (4, 6, 512)
attn2 (4, 6, 512)
ffn_output (4, 6, 512)
out3 (4, 6, 512)
normed_x (4, 6, 512)
attn1 (4, 6, 512)
attn2 (4, 6, 512)
ffn_output (4, 6, 512)
out3 (4, 6, 512)
normed_x (4, 6, 512)
attn1 (4, 6, 512)
attn2 (4, 6, 512)
ffn_output (4, 6, 512)
out3 (4, 6, 512)
dec_output (4, 583, 512)
final_output (4, 6, 258)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 ... 1.2616850e-02
   1.1365835e-02 1.0914088e

enc_output (4, 583, 512)
target :  (4, 9, 258)
normed_x (4, 9, 512)
attn1 (4, 9, 512)
attn2 (4, 9, 512)
ffn_output (4, 9, 512)
out3 (4, 9, 512)
normed_x (4, 9, 512)
attn1 (4, 9, 512)
attn2 (4, 9, 512)
ffn_output (4, 9, 512)
out3 (4, 9, 512)
normed_x (4, 9, 512)
attn1 (4, 9, 512)
attn2 (4, 9, 512)
ffn_output (4, 9, 512)
out3 (4, 9, 512)
normed_x (4, 9, 512)
attn1 (4, 9, 512)
attn2 (4, 9, 512)
ffn_output (4, 9, 512)
out3 (4, 9, 512)
dec_output (4, 583, 512)
final_output (4, 9, 258)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 ... 1.2616850e-02
   1.1365835e-02 1.0914088e

enc_output (4, 583, 512)
target :  (4, 12, 258)
normed_x (4, 12, 512)
attn1 (4, 12, 512)
attn2 (4, 12, 512)
ffn_output (4, 12, 512)
out3 (4, 12, 512)
normed_x (4, 12, 512)
attn1 (4, 12, 512)
attn2 (4, 12, 512)
ffn_output (4, 12, 512)
out3 (4, 12, 512)
normed_x (4, 12, 512)
attn1 (4, 12, 512)
attn2 (4, 12, 512)
ffn_output (4, 12, 512)
out3 (4, 12, 512)
normed_x (4, 12, 512)
attn1 (4, 12, 512)
attn2 (4, 12, 512)
ffn_output (4, 12, 512)
out3 (4, 12, 512)
dec_output (4, 583, 512)
final_output (4, 12, 258)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 ... 1.2616850e-02
   1.

...

enc_output (4, 583, 512)
target :  (4, 576, 258)
normed_x (4, 576, 512)
attn1 (4, 576, 512)
attn2 (4, 576, 512)
ffn_output (4, 576, 512)
out3 (4, 576, 512)
normed_x (4, 576, 512)
attn1 (4, 576, 512)
attn2 (4, 576, 512)
ffn_output (4, 576, 512)
out3 (4, 576, 512)
normed_x (4, 576, 512)
attn1 (4, 576, 512)
attn2 (4, 576, 512)
ffn_output (4, 576, 512)
out3 (4, 576, 512)
normed_x (4, 576, 512)
attn1 (4, 576, 512)
attn2 (4, 576, 512)
ffn_output (4, 576, 512)
out3 (4, 576, 512)
dec_output (4, 583, 512)
final_output (4, 576, 258)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 .

enc_output (4, 583, 512)
target :  (4, 579, 258)
normed_x (4, 579, 512)
attn1 (4, 579, 512)
attn2 (4, 579, 512)
ffn_output (4, 579, 512)
out3 (4, 579, 512)
normed_x (4, 579, 512)
attn1 (4, 579, 512)
attn2 (4, 579, 512)
ffn_output (4, 579, 512)
out3 (4, 579, 512)
normed_x (4, 579, 512)
attn1 (4, 579, 512)
attn2 (4, 579, 512)
ffn_output (4, 579, 512)
out3 (4, 579, 512)
normed_x (4, 579, 512)
attn1 (4, 579, 512)
attn2 (4, 579, 512)
ffn_output (4, 579, 512)
out3 (4, 579, 512)
dec_output (4, 583, 512)
final_output (4, 579, 258)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 .

enc_output (4, 583, 512)
target :  (4, 582, 258)
normed_x (4, 582, 512)
attn1 (4, 582, 512)
attn2 (4, 582, 512)
ffn_output (4, 582, 512)
out3 (4, 582, 512)
normed_x (4, 582, 512)
attn1 (4, 582, 512)
attn2 (4, 582, 512)
ffn_output (4, 582, 512)
out3 (4, 582, 512)
normed_x (4, 582, 512)
attn1 (4, 582, 512)
attn2 (4, 582, 512)
ffn_output (4, 582, 512)
out3 (4, 582, 512)
normed_x (4, 582, 512)
attn1 (4, 582, 512)
attn2 (4, 582, 512)
ffn_output (4, 582, 512)
out3 (4, 582, 512)
dec_output (4, 583, 512)
final_output (4, 582, 258)
input :  (4, 583, 129)
x :  tf.Tensor(
[[[9.0760412e-03 7.9445848e-03 5.2796793e-03 ... 5.6781102e-04
   1.8663406e-04 4.5797098e-04]
  [5.0227474e-02 3.4398895e-02 1.3238299e-02 ... 1.6703292e-03
   9.0455939e-04 1.3974305e-04]
  [3.5573345e-02 3.0037913e-02 1.1705405e-02 ... 2.7970169e-03
   1.9327636e-03 9.0092799e-04]
  ...
  [3.4712257e-03 5.0844789e-02 8.4884584e-02 ... 1.4801581e-02
   1.5439548e-02 1.7142929e-02]
  [6.6102281e-02 1.1601095e-01 1.7200373e-01 .

c:\users\qkrwo\anaconda3\envs\nlp_task\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: scipy.zeros is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros instead


SystemExit: 0

In [78]:
result[0][0]

<tf.Tensor: shape=(258,), dtype=float32, numpy=
array([0.00000000e+00, 2.42580418e-02, 1.12882458e-01, 0.00000000e+00,
       5.90921938e-02, 0.00000000e+00, 4.58631665e-02, 0.00000000e+00,
       0.00000000e+00, 1.38342842e-01, 1.95561409e-01, 1.32136583e-01,
       3.81718755e-01, 3.69749844e-01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.07864529e-01, 1.72209978e-01, 0.00000000e+00,
       0.00000000e+00, 9.77572128e-02, 6.30682185e-02, 1.45345926e-01,
       7.36483857e-02, 1.61640108e-01, 4.40682704e-03, 0.00000000e+00,
       3.30893062e-02, 5.64189814e-02, 5.87175637e-02, 0.00000000e+00,
       6.02256395e-02, 7.21756509e-03, 6.61499202e-02, 4.39364649e-02,
       8.29221122e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.07097004e-02, 0.00000000e+00, 1.64335668e-02, 1.10212043e-01,
       7.86919240e-03, 0.00000000e+00, 2.16858163e-02, 1.45288603e-02,
       0.00000000e+00, 2.12304965e-02, 5.93296066e-02, 0.00000000e+00,
       0.00000000e+00, 2.7642